In [22]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [23]:
%load_ext dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [24]:
import os

In [25]:
!pwd
!ls -l ./

/mnt/datadrive01/prj/AzureChestXRayNoAML/code
total 3232
-rwxrwxrwx 1 loginVM_001 loginVM_001  401244 Sep 24 01:38 00_create_docker_image.html
-rwxrwxrwx 1 loginVM_001 loginVM_001 2621340 Sep 24 05:20 00_create_docker_image.ipynb
drwxrwxrwx 3 loginVM_001 loginVM_001    4096 Sep 24 05:12 01_DataPrep
drwxrwxrwx 3 loginVM_001 loginVM_001    4096 Sep 24 05:18 02_Model
drwxrwxrwx 2 loginVM_001 loginVM_001    4096 Sep 24 04:24 03_Deployment
-rwxrwxrwx 1        1000 users        255403 Sep 23 23:56 edit_python_files.html
-rwxrwxrwx 1        1000 users          6602 Sep 23 23:58 edit_python_files.ipynb
drwxrwxrwx 3        1000 users          4096 Sep 24 02:07 notShared
drwxrwxrwx 3 loginVM_001 loginVM_001    4096 Sep 23 23:10 src


In [26]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/chestxray-no-aml-gpu"

Overwriting .env


In [36]:
%%writefile ./../docker/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh 
        
#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
# https://www.microsoft.com/net/download/linux-package-manager/ubuntu16-04/runtime-2.1.2
RUN wget -q https://packages.microsoft.com/config/ubuntu/16.04/packages-microsoft-prod.deb && \
    dpkg -i packages-microsoft-prod.deb && \ 
    apt-get install  -y --no-install-recommends apt-transport-https && \
    apt-get update && \
    apt-get install  -y --no-install-recommends aspnetcore-runtime-2.1  && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
ENV NB_USER keras
ENV NB_UID 1000

RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
    chown $NB_USER $CONDA_DIR -R && \
    mkdir -p /src && \
    chown $NB_USER /src

USER $NB_USER

ARG python_version=3.6

# tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow-gpu \
      tensorflow-tensorboard \
      matplotlib \
      numpy==1.14.5 && \
    conda install \
      bcolz \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      requests \
      scikit-learn \
      six \
      tqdm && \
    conda install -c conda-forge imageio && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    pip install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl \
      torchvision && \
    conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile


In [37]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':1.0.0'
docker_file_location = os.path.join(*(['.', '..','docker', 'dockerfile']))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu:1.0.0'

'./../docker/dockerfile'

In [38]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu:1.0.0 -f ./../docker/dockerfile .
Sending build context to Docker daemon  11.21MB
Step 1/16 : FROM nvidia/cuda:9.0-cudnn7-devel
 ---> 288866c2c06c
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> bc9e7c9ac9dd
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 3c415595ae4e
Step 4/16 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> e08bdb87f797
Step 5/16 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> 3e2afc791489
Step 6/16 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" |

In [39]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu]

4e886f84: Preparing 
a89b6996: Preparing 
a1f3e5d0: Preparing 
9618113d: Preparing 
e517b5f3: Preparing 
e5d9d370: Preparing 
6d4f5f2e: Preparing 
3c790502: Preparing 
ac05da10: Preparing 
19f38a85: Preparing 
d6639b96: Preparing 
9e19929c: Preparing 
b2f378bb: Preparing 
afdbe580: Preparing 
43c86cbc: Preparing 


e886f84: Pushing  811.7MB/5.587GB12APushing  242.1MB/5.587GBPushing  440.3MB/5.587GBPushing  642.1MB/5.587GB

e886f84: Pushing  1.769GB/5.587GBPushing   1.52GB/5.587GBPushing  1.585GB/5.587GBPushing  1.685GB/5.587GB

e886f84: Pushing  2.502GB/5.587GBPushing  2.083GB/5.587GB

e886f84: Pushing  3.207GB/5.587GBPushing  2.852GB/5.587GBPushing  3.137GB/5.587GB

e886f84: Pushing  4.122GB/5.587GBPushing  3.938GB/5.587GBPushing  3.993GB/5.587GB

e886f84: Pushing  4.909GB/5.587GBPushing  4.509GB/5.587GB

e886f84: Pushing  5.583GB/5.587GBPushing   4.92GB/5.587GBPushing  4.921GB/5.587GBPushing  5.564GB/5.587GB

e886f84: Pushed    5.68GB/5.587GBPushing  5.643GBPushing  5.667GB1.0.0: digest: sha256:8b1ccad67677d72b687c998cbd9e9a9ab6c329c57cafbb9daf4cb6fc164f76a7 size: 3688


In [31]:
!echo sudo nvidia-docker run -i -t -p 10003:8888 -v '$(pwd)':/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0 /bin/bash -c '"/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip='*' --port=8888 --no-browser --allow-root"'

sudo nvidia-docker run -i -t -p 10003:8888 -v $(pwd):/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0 /bin/bash -c "/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip=* --port=8888 --no-browser --allow-root"


In [40]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 267240 bytes to 00_create_docker_image.html
